In [5]:
from hmmlearn import hmm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
from collections import Counter

In [6]:
X = pd.DataFrame(pd.pandas.read_csv("../data/3Boxes_coords.csv", names=['Gauche_head', 'Haut_head', 'Largeur_head', 'Longeur_head', 'Gauche_snout', 'Haut_snout', 'Largeur_snout', 'Longeur_snout', 'Gauche_cow', 'Haut_cow', 'Largeur_cow', 'Longeur_cow']))
X = X.drop(index=X.index[0])
model_3boxes = joblib.load('models/EM/hmm_model_head_snout_cow.pkl')
hidden_states = model_3boxes.predict(X)
hidden_states_count = pd.DataFrame(hidden_states, columns=['values'])
# print(hidden_states_count)
hidden_states_counted = hidden_states_count.groupby([hidden_states_count['values'].ne(hidden_states_count['values'].shift()).cumsum(), 'values']).size()

data = {
    'state': [0, 3, 2, 1, 2, 1, 2],
    'count': [484, 61, 111, 23, 187, 464, 45]
}
df = pd.DataFrame(data)

df['frame'] = df['count'].cumsum()
df['seconds'] = df['frame']/30

print(df)

   state  count  frame    seconds
0      0    484    484  16.133333
1      3     61    545  18.166667
2      2    111    656  21.866667
3      1     23    679  22.633333
4      2    187    866  28.866667
5      1    464   1330  44.333333
6      2     45   1375  45.833333


In [7]:
behavior = pd.pandas.read_csv("E:/UQAM/INF889E/projet/Recherche/Data/emotion_analyses/emotion_analyses/NOT_Lou_annotations.csv", header=None)

new_columns = behavior[0].str.split(',', expand=True)

behavior[new_columns.columns] = new_columns

behavior = behavior.drop(columns=[0])
temps_annotation = behavior[[5, 11]].drop(behavior.index[0])
annotation = pd.DataFrame(temps_annotation)
annotation = annotation.replace("00:00:", "", regex=True)
X_annotation = annotation.values

#print(str(annotation[5][1]) <= '00')
print(X_annotation)

[['00' 'No Object']
 ['00' 'Eating/drinking']
 ['00' 'No']
 ['00' 'No object']
 ['00' 'Ears not visible']
 ['00' 'No objects']
 ['04' 'Object STARTS descent']
 ['04' 'No object']
 ['04' 'Object not in reach']
 ['04' 'No objects']
 ['04' 'Not looking at object']
 ['06' 'Object not in reach']
 ['06' '>2 heads']
 ['06' 'No Object']
 ['06' 'No interaction']
 ['07' '>2 heads']
 ['07' '1 to 2 heads']
 ['07' 'Not looking at object']
 ['07' 'Looking at object']
 ['08' 'Ears not visible']
 ['08' 'Facing forward']
 ['11' '1 to 2 heads']
 ['11' '>2 heads']
 ['11' 'Facing forward']
 ['11' 'Other ear position']
 ['15' 'Other ear position']
 ['15' 'Facing forward']
 ['16' '>2 heads']
 ['16' '1 to 2 heads']
 ['17' 'Eating/drinking']
 ['17' 'Standing up']
 ['17' 'No interaction']
 ['17' 'Move away/stand up']
 ['18' 'Facing forward']
 ['18' 'Other ear position']
 ['19' '1 to 2 heads']
 ['19' '>2 heads']
 ['20' 'Standing up']
 ['20' 'Resting/inactive']
 ['20' 'Other ear position']
 ['20' 'Facing forward

In [8]:
best_score = -float('inf')
best_model = None
best_n_component = 0

for n_component in range(2, 5):
    model_X = hmm.MultinomialHMM(n_components=n_component, random_state=50, n_iter=50)
    model_X.fit(X_annotation)
    score_X = model_X.score(X_annotation)
    if score_X > best_score:
        best_score = score_X
        best_model = model_X
        best_n_component = n_component

Z_count = best_model.predict(X_annotation)
states = pd.unique(Z_count)
states

MultinomialHMM has undergone major changes. The previous version was implementing a CategoricalHMM (a special case of MultinomialHMM). This new implementation follows the standard definition for a Multinomial distribution (e.g. as in https://en.wikipedia.org/wiki/Multinomial_distribution). See these issues for details:
https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340


ValueError: could not convert string to float: 'No Object'

In [4]:
# Ethogram
novel_object = ["Approach" "Move away/standing up", "Sniff", "Push/bite", "Object starts descent", "Object ends descent", "No object", "No interaction"]
normal_behaviours = ["Sleeping", "Eating/Drinking", "Exploration", "Social Interaction", "Resting/Inactive", "Standing up", "Lying down", "Scratching/Grooming", "Other"]
Rumination = ["Yes", "No", "Not visible"]
Distances = ["< 1 head", "1 to 2 heads", "> 2 heads", "Stretched maximum", "No object"] 
Ear_Position = ["Facing forward", "Ears not visible", "Other ear position"]
Other_behaviours = ["Looking at object", "Not looking at object", "No object"] 

In [5]:
count_detail = []
behavior = []
j = 0

for i in range(len(df)):
    temp_list = []
    while j < len(annotation):
        j+=1
        if str(annotation[5][j]) <= str(df['seconds'].iloc[i]):
            temp_list.append(annotation[11][j])
        else:
            count = Counter(temp_list)
            count_dict = dict(count)
            count_detail.append(count)
                        
            grouped_counts = {
                "novel_object": sum(count.get(word, 0) for word in novel_object),
                "normal_behaviours": sum(count.get(word, 0) for word in normal_behaviours),
                "Rumination": sum(count.get(word, 0) for word in Rumination),
                "Distances": sum(count.get(word, 0) for word in Distances),
                "Ear_Position": sum(count.get(word, 0) for word in Ear_Position),
                "Other_behaviours": sum(count.get(word, 0) for word in Other_behaviours)
            }
            behavior.append(grouped_counts)

            break
        

count_detail_df = pd.DataFrame(count_detail)
behavior_df = pd.DataFrame(behavior)

df = pd.concat([df, count_detail_df, behavior_df], axis=1)


print(df)



   state  count  frame    seconds  No Object  Eating/drinking   No  No object  \
0      0    484    484  16.133333        2.0              1.0  1.0        2.0   
1      3     61    545  18.166667        NaN              NaN  NaN        NaN   
2      2    111    656  21.866667        NaN              NaN  1.0        NaN   
3      1     23    679  22.633333        NaN              NaN  NaN        NaN   
4      2    187    866  28.866667        NaN              NaN  NaN        NaN   
5      1    464   1330  44.333333        NaN              NaN  NaN        NaN   
6      2     45   1375  45.833333        NaN              NaN  NaN        NaN   

   Ears not visible  No objects  ...  Move away/stand up  Resting/inactive  \
0               2.0         2.0  ...                 NaN               NaN   
1               NaN         NaN  ...                 1.0               NaN   
2               1.0         NaN  ...                 1.0               1.0   
3               NaN         NaN  ...   

In [6]:
df.to_csv("../INF889E Project/Trained data/behavior_3box_hmm.csv")

PermissionError: [Errno 13] Permission denied: '../INF889E Project/Trained data/behavior_3box_hmm.csv'